3.1.7, 3.3.2, 3.9.2

## 3.1.7
$n = 6 \\
a_{ij} = \frac{256}{(5+c_{ij}\cdot0.256)^5}$

In [1]:
import numpy as np
from numpy.linalg import solve, cond, norm
import copy

## 1.

In [4]:
n = 6
N = 7
def c(i, j):
    return 0.1*N*i*j

def a(i, j):
    return 256/(5+c(i,j)*0.256)**5

In [5]:
A = np.array([[a(i,j) for j in range(1, n+1)] for i in range(1, n+1)]).reshape((n, n))
b = np.array([N]*n)

In [6]:
x_prec = solve(A, b)
x_prec

array([  1715.381414  , -12130.10702664,  40418.13434764, -68845.50948181,
        57944.4604052 , -19088.68444808])

Проверка:

In [7]:
A@x_prec

array([7., 7., 7., 7., 7., 7.])

## 2.

In [8]:
cnd = cond(A, p=np.inf)
cnd

3998997.78887232

## 3.

In [9]:
def b_i(b, i, delta=0.001):
    d = np.array([0]*b.size, dtype=float)
    d[i] = delta
    return b + d

In [10]:
def d_ch(A, x1, b1):
    _x = solve(A, b1)
    return norm(x1 - _x, ord=np.inf)/norm(x1, ord=np.inf)

In [11]:
d = [d_ch(A, x_prec, b_i(b, i)) for i in range(0, n)]

In [12]:
d

[0.000645515563417822,
 0.007762366810405227,
 0.03420153558329723,
 0.07017354787599629,
 0.06717102644995922,
 0.024316852067923035]

## 4.
Наибольшую погрешность даёт четвертая компонента вектора b (индекс = 3)

## 5.

In [13]:
b_3 = b_i(b, 3)
_x_3 = solve(A, b_3)
_x_3

array([  1757.11786592, -12655.48450671,  42772.75648994, -73676.64313748,
        62568.88394356, -20762.79051769])

In [14]:
d_b_3 = norm(b - b_3, ord=np.inf)/norm(b, ord=np.inf)
d_x_3 = norm(x_prec - _x_3, ord=np.inf)/norm(x_prec, ord=np.inf)

In [15]:
cnd*d_b_3

571.2853984105222

Теоритическая погрешность очень велика из-за того, что исходная матрица A плохо обусловлена. Значит, даже при незначительной относительной погрешности $\delta (b')$, где $b' = Ax'$ ($x'$ - это приближенный результат), решение сестемы может значительно отличаться от истинного.

## 3.3.2

In [16]:
A = np.array([[3, 1, 0, 0, 0],
              [1, 2, 1, 0, 0],
              [0, 1, 1, 1, 0],
              [0, 0, 1, 0, 1],
              [0, 0, 0, 1, 1]], dtype=np.float32).reshape(5, 5)

In [17]:
def b_set_func(i, n=5):
  temp = np.array([0]*n)
  temp[i] = np.random.randint(100)+1
      
  print(temp)
  return temp

k=5
b_set = [b_set_func(i) for i in range(k)]
x_set = [solve(A, b_set[i]) for i in range(k)]
norm_A_rev = np.max([norm(x_set[i])/norm(b_set[i]) for i in range(k)])
norm_A = norm(A, ord=np.inf)

[29  0  0  0  0]
[ 0 83  0  0  0]
[0 0 5 0 0]
[ 0  0  0 77  0]
[ 0  0  0  0 96]


In [18]:
norm_A*norm_A_rev

5.268311118453079

In [19]:
cond(A)

7.5435047

## 3.9.2

$n=20 \\
m=8 \\
b_{i} = 200+50*i$

$A_{i, j} = 
\begin{cases} 
\frac{i+j}{m+n}, & \mbox{if } i \ne j \\
n + m^2 + \frac{j}{m} + \frac{i}{n}, & \mbox{if } i = j
\end{cases} \\
i, j = 1 \dots n $

In [20]:
def chol(a):
  
  
    a = copy.deepcopy(a)
    
    m=a.shape[0]
    

    a[0, 0] = np.sqrt(a[0, 0])
    for i in range(1, m):
        a[i, 0] = a[i, 0] / a[0, 0]
     
    for k in range(1, m-1):
        a[k, k] = (a[k, k] - sum([a[k, q]**2 for q in range(0, k)]))**0.5
        for i in range(k+1, m):
            a[i, k] = (a[i, k] - sum([a[i, q]*a[k, q] for q in range(0, k)]))/a[k, k]
          
    a[m-1, m-1] = (a[m-1, m-1] - sum([a[m-1, q]**2 for q in range(0, m-1)]))**0.5
    
    for i in range(m):
        for j in range(i+1, m):
            a[i, j] = 0

    return a
        

In [21]:
n=20
m=8
def Aij(i, j, m=m, n=n):
    if i == j:
        return n + m**2 + j/m + i/n
    else:
        return (i+j)/(m+n)
def bi(i):
    return 200 + 50*i

In [22]:
A = np.array([[Aij(i, j) for i in range(1, n+1)] for j in range(1, n+1)]).reshape(n, n)
b = np.array([bi(i) for i in range(1, n+1)])
L = chol(A)
LT = L.T
y = solve(L, b)
x = solve(LT, y)

In [23]:
x

array([ 2.13216095,  2.6630651 ,  3.19266488,  3.72096508,  4.24797048,
        4.77368584,  5.29811588,  5.82126532,  6.34313883,  6.86374109,
        7.38307673,  7.90115037,  8.4179666 ,  8.93352999,  9.4478451 ,
        9.96091645, 10.47274855, 10.98334589, 11.49271292, 12.00085408])

In [24]:
cond(A)

1.2144435476138873

In [25]:
norm_A_rev = norm(x, ord=np.inf)/norm(b, ord=np.inf)
norm_A = norm(A, ord=np.inf)

In [26]:
norm_A*norm_A_rev

1.0786481942801718